In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [21]:
cities_file = "Resources/cities.csv"
cities_df = pd.read_csv(cities_file)
new_cities_df = cities_df[['id', 'name', 'country']].copy()
new_cities_df = new_cities_df.rename(columns={'id':'city_id'})
new_cities_df.head()


,city_id,name,country
0,5,Aberdeen,Scotland
1,6,Adelaide,Australia
2,7,Algiers,Algeria
3,9,Ankara,Turkey
4,16,Belém,Brazil


In [20]:
# stations_file = "Resources/stations.csv"
# stations_df = pd.read_csv(stations_file)
# new_stations_df = stations_df[['city_id', 'id','name']].copy()
# new_stations_df.head()


In [33]:
tracks_file = "Resources/tracks.csv"
tracks_df = pd.read_csv(tracks_file)
new_tracks_df = tracks_df[['city_id','length']].copy()
track_length = new_tracks_df.groupby('city_id').sum()
track_length.reset_index(inplace=True)
# new_tracks_df.head()
track_length.head()


,city_id,length
0,1,1472481
1,4,475631
2,14,630124
3,15,685994
4,19,1231


In [31]:
track_length_df = new_cities_df.merge(track_length, on='city_id')
track_length_df.sort_values(by='name', inplace=True)
track_length_df.reset_index(inplace=True, drop=True)
track_length_df.head(30)

,city_id,name,country,length
0,275,Angers,France,33944
1,14,Barcelona,Spain,630124
2,15,Beijing,China,685994
3,19,Berlin,Germany,1231
4,283,Besancon,France,28824
5,258,Bilbao,Spain,62962
6,21,Bogotá,Colombia,173514
7,23,Bordeaux,France,129963
8,139,Boston,United States,724935
9,293,Brest,France,28902
